In [1]:
import os
import requests
import dotenv
import numpy as np
import pandas as pd
import alpaca_trade_api as alpaca_api
import ccxt

%matplotlib inline

In [2]:
dotenv.load_dotenv()

True

In [3]:
alpaca_api_key = os.getenv("alpaca_api_key")
alpaca_secret_key = os.getenv("alpaca_secret_key")

print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpaca Secret Key type: {type(alpaca_secret_key)}")

Alpaca Key type: <class 'str'>
Alpaca Secret Key type: <class 'str'>


In [4]:
alpaca = alpaca_api.REST(alpaca_api_key, alpaca_secret_key, api_version="v2")

In [5]:
faangmula_tickers = ["FB", "AMZN", "AAPL", "NFLX", "GOOG", "MSFT", "UBER", "LYFT", "ABNB"]
start = pd.Timestamp("2020-07-14", tz="America/New_York").isoformat()
end = pd.Timestamp("2020-08-14", tz="America/New_York").isoformat()

faangmula_original = alpaca.get_bars(faangmula_tickers, alpaca_api.TimeFrame.Day, start = start, end = end).df
faangmula_original

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2020-07-14 04:00:00+00:00,379.50,389.02,375.510,388.23,42751858,475871,383.301051,AAPL
2020-07-15 04:00:00+00:00,395.98,396.99,385.960,390.90,38299588,437745,391.194769,AAPL
2020-07-16 04:00:00+00:00,386.24,389.62,383.620,386.09,27645834,345198,385.872228,AAPL
2020-07-17 04:00:00+00:00,388.00,388.59,383.360,385.31,23046755,257104,385.932761,AAPL
2020-07-20 04:00:00+00:00,385.60,394.00,384.250,393.43,22579507,281654,389.779094,AAPL
...,...,...,...,...,...,...,...,...
2020-08-10 04:00:00+00:00,33.00,33.18,31.510,32.27,21347585,123516,32.199165,UBER
2020-08-11 04:00:00+00:00,31.67,32.21,30.650,31.21,31569923,176040,31.413623,UBER
2020-08-12 04:00:00+00:00,31.46,32.22,30.690,30.84,31514151,136705,31.330652,UBER


In [6]:
faangmula = faangmula_original.drop(['open', 'high', 'low', 'volume', 'trade_count', 'vwap'], axis=1).pivot(columns = 'symbol')
faangmula

close                                           \
symbol                       AAPL     AMZN      FB     GOOG   LYFT    MSFT   
timestamp                                                                    
2020-07-14 04:00:00+00:00  388.23  3084.00  239.73  1520.58  27.92  208.35   
2020-07-15 04:00:00+00:00  390.90  3008.87  240.28  1513.64  30.78  208.04   
2020-07-16 04:00:00+00:00  386.09  2999.90  240.93  1518.00  29.58  203.92   
2020-07-17 04:00:00+00:00  385.31  2961.97  242.03  1515.55  29.73  202.88   
2020-07-20 04:00:00+00:00  393.43  3196.84  245.42  1565.72  30.57  211.60   
2020-07-21 04:00:00+00:00  388.00  3138.29  241.75  1558.42  30.49  208.75   
2020-07-22 04:00:00+00:00  389.09  3099.91  239.87  1568.49  30.84  211.75   
2020-07-23 04:00:00+00:00  371.38  2986.55  232.60  1515.68  30.86  202.54   
2020-07-24 04:00:00+00:00  370.46  3008.91  230.71  1511.87  30.42  201.30   
2020-07-27 04:00:00+00:00  379.24  3055.21  233.50  1530.20  29.89  203.85   
2020-07-28 04:00:00+00:00  373.01  3000.33  230.12  1500.34  29.49  202.02   
2020-07-29 04:00:00+00:00  380.16  3033.53  233.29  1522.02  29.89  204.06   
2020-07-30 04:00:00+00:00  384.76  3051.88  234.50  1531.45  29.48  203.90   
2020-07-31 04:00:00+00:00  425.04  3164.68  253.67  1482.96  29.23  205.01   
2020-08-03 04:00:00+00:00  435.75  3111.89  251.96  1474.45  29.09  216.54   
2020-08-04 04:00:00+00:00  438.66  3138.83  249.83  1464.97  30.50  213.29   
2020-08-05 04:00:00+00:00  440.25  3205.03  249.12  1473.61  30.89  212.94   
2020-08-06 04:00:00+00:00  455.61  3225.00  265.28  1500.10  32.34  216.35   
2020-08-07 04:00:00+00:00  444.45  3167.46  268.44  1494.49  30.19  212.48   
2020-08-10 04:00:00+00:00  450.91  3148.16  263.00  1496.10  31.05  208.25   
2020-08-11 04:00:00+00:00  437.50  3080.67  256.13  1480.32  30.65  203.38   
2020-08-12 04:00:00+00:00  452.04  3162.24  259.89  1506.62  30.52  209.19   
2020-08-13 04:00:00+00:00  460.04  3161.02  261.30  1518.45  28.88  208.70   
2020-08-14 04:00:00+00:00  459.63  3148.02  261.24  1507.73  28.51  208.90   

                                          
symbol                       NFLX   UBER  
timestamp                                 
2020-07-14 04:00:00+00:00  524.88  30.93  
2020-07-15 04:00:00+00:00  523.26  32.81  
2020-07-16 04:00:00+00:00  527.39  32.35  
2020-07-17 04:00:00+00:00  492.99  32.55  
2020-07-20 04:00:00+00:00  502.41  33.66  
2020-07-21 04:00:00+00:00  490.10  32.55  
2020-07-22 04:00:00+00:00  489.82  32.66  
2020-07-23 04:00:00+00:00  477.58  32.17  
2020-07-24 04:00:00+00:00  480.45  31.18  
2020-07-27 04:00:00+00:00  495.65  30.99  
2020-07-28 04:00:00+00:00  488.51  30.81  
2020-07-29 04:00:00+00:00  484.48  31.02  
2020-07-30 04:00:00+00:00  485.80  30.24  
2020-07-31 04:00:00+00:00  488.88  30.26  
2020-08-03 04:00:00+00:00  498.62  31.19  
2020-08-04 04:00:00+00:00  509.64  32.68  
2020-08-05 04:00:00+00:00  502.11  33.20  
2020-08-06 04:00:00+00:00  509.08  34.71  
2020-08-07 04:00:00+00:00  494.73  32.90  
2020-08-10 04:00:00+00:00  483.38  32.27  
2020-08-11 04:00:00+00:00  466.93  31.21  
2020-08-12 04:00:00+00:00  475.47  30.84  
2020-08-13 04:00:00+00:00  481.33  30.46  
2020-08-14 04:00:00+00:00  482.68  29.99

In [7]:
def random_portfolios_generator(tickers, n):
    random_portfolios = pd.DataFrame(tickers, columns = ['ticker'])
    for n in range(0, n):
        random_portfolios[f'Random Portfolio {n+1}'] = [np.random.rand() for i in range (0, 9)]
        random_portfolios[f'Random Portfolio {n+1}'] = random_portfolios[f'Random Portfolio {n+1}'] / sum(random_portfolios[f'Random Portfolio {n+1}'])
    return(random_portfolios)